In [194]:
import requests
import pandas as pd
from seleniumwire import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as BS
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities    
from selenium.webdriver.chrome.options import Options 
import configparser
import re
from datetime import timedelta, datetime
import time
import sys
from dateutil import relativedelta, parser, rrule
from dateutil.rrule import WEEKLY


* need to add the .ini option
* add the time pull
* add the custom date pull
* format the key data appropriately
* add the activities, sleep and hr data functions too

In [252]:
class whoop_login:
    '''A class object to allow a user to login and store their authorization code, 
        then perform pulls using the code in order to access different types of data'''
    
    def __init__(self, auth_code=None, whoop_id=None,current_datetime=datetime.utcnow()):
        self.auth_code=auth_code
        self.whoop_id=whoop_id
        self.current_datetime=current_datetime
        self.start_datetime=None
        self.all_data=None
        self.all_activities=None
        self.sport_dict=None
        
    
    def pull_api(self, url,df=False):
        auth_code=self.auth_code
        headers={'authorization':auth_code}
        pull=requests.get(url,headers=headers)
        if pull.status_code==200 and len(pull.content)>1:
            if df:
                d=pd.json_normalize(pull.json())
                return d
            else:
                return pull.json()
        else:
            return "no response"   
    
    def get_authorization(self,user_ini=False):
         
        def get_button():
            work=0
            while work==0:
                try: 
                    back=browser.find_element_by_class_name('datepicker--prev')
                    back.click()
                    work=1
                except:
                    work=0
                    
        if not user_ini:
            url='https://app.whoop.com/login'
            browser=webdriver.Chrome('/usr/local/bin/chromedriver')
            browser.get(url)
            
            try:
                WebDriverWait(browser, 100).until(EC.url_changes(url))
            except TimeoutException:
                print("The script timed out while waiting for login. Please try again")
    
            try:
            ## Getting the authorization code
                whoop_requests=[request for request in browser.requests if 'api' and 'whoop' in request.url]
                authorization_code=whoop_requests[0].headers['authorization']
            except :
                current_url=browser.current_url
                get_button()
                work=0
                while work<100:
                    try:
                        WebDriverWait(browser, 100).until(EC.url_changes(current_url))
                        whoop_requests=[request for request in browser.requests if 'api' and 'whoop' and 'cycles' in request.url]
                        authorization_code=whoop_requests[0].headers['authorization']
                        work=101
                    except:
                        work+=1
                if work==100:
                    sys.exit()
                    print("The script timed out while waiting for login. Please try again")
                
            self.auth_code=authorization_code
            
            ## Getting the whoop id
            profile_requests=[request for request in browser.requests if 'users' and 'profile' in request.url]
            try:
                athlete_id=re.search('(?<=users/)(\d+)(?=\?include)',profile_requests[0].url).group(1)
            except AttributeError:
                print('not found')
            self.whoop_id=athlete_id
            
            profile_url='https://api-7.whoop.com/users/{}?include=profile&include=teams'.format(athlete_id)
            start_datetime=self.pull_api(profile_url,df=False)['profile']['createdAt']
            self.start_datetime=start_datetime
        else:
            config=configparser.ConfigParser()
            config.read(user_ini)

            url='https://app.whoop.com/login'
            chrome_options = Options()  
            chrome_options.add_argument("--headless") 
            browser=webdriver.Chrome('/usr/local/bin/chromedriver',chrome_options=chrome_options)
            browser.get(url)
            
            username=browser.find_element_by_name('username')
            password=browser.find_element_by_name('password')
            username.send_keys(config['whoop']['username'])
            password.send_keys(config['whoop']['password'])
            submit=browser.find_element_by_name('form')
            submit.click()
            
            try:
                WebDriverWait(browser, 100).until(EC.url_changes(url))
            except TimeoutException:
                print("The script timed out while waiting for login. Please try again")
    
            try:
            ## Getting the authorization code
                whoop_requests=[request for request in browser.requests if 'api' and 'whoop' in request.url]
                authorization_code=whoop_requests[0].headers['authorization']
            except :
                current_url=browser.current_url
                get_button()
                work=0
                while work<100:
                    try:
                        WebDriverWait(browser, 100).until(EC.url_changes(current_url))
                        whoop_requests=[request for request in browser.requests if 'api' and 'whoop' and 'cycles' in request.url]
                        authorization_code=whoop_requests[0].headers['authorization']
                        work=101
                    except:
                        work+=1
                if work==100:
                    sys.exit()
                    print("The script timed out while waiting for login. Please try again")
                
            self.auth_code=authorization_code
            
            ## Getting the whoop id
            profile_requests=[request for request in browser.requests if 'users' and 'profile' in request.url]
            try:
                athlete_id=re.search('(?<=users/)(\d+)(?=\?include)',profile_requests[0].url).group(1)
            except AttributeError:
                print('not found')
            self.whoop_id=athlete_id
            
            profile_url='https://api-7.whoop.com/users/{}?include=profile&include=teams'.format(athlete_id)
            start_datetime=self.pull_api(profile_url,df=False)['profile']['createdAt']
            self.start_datetime=start_datetime
        
    
    def get_keydata_all(self):
        '''
        This function returns a dataframe of WHOOP metrics for each day of WHOOP membership. 
        In the resulting dataframe, each day is a row and contains strain, recovery, and sleep information
        '''
        
        if self.start_datetime:
            if self.all_data is not None:
                ## All data already pulled
                return self.all_data
            else:
                start_date=parser.isoparse(client.start_datetime).replace(tzinfo=None)
                end_time='T23:59:59.999Z'
                start_time='T00:00:00.000Z'
                intervals=rrule.rrule(freq=WEEKLY,interval=1,until=self.current_datetime, dtstart=start_date)
                date_range=[[d.strftime('%Y-%m-%d') + start_time,
                            (d+relativedelta.relativedelta(weeks=1)).strftime('%Y-%m-%d') + end_time] for d in intervals]
                all_data=pd.DataFrame()
                for dates in date_range:
                    cycle_url='https://api-7.whoop.com/users/{}/cycles?end={}&start={}'.format(self.whoop_id,
                                                                                           dates[1],
                                                                                           dates[0])
                    data=self.pull_api(cycle_url,df=True)
                    all_data=pd.concat([all_data,data])
                all_data.reset_index(drop=True,inplace=True)

                ## fixing the day column so it's not a list
                all_data['days']=all_data['days'].map(lambda d: d[0])
                all_data.rename(columns={"days":'day'},inplace=True)

                ## Putting all time into minutes instead of milliseconds
                sleep_cols=['qualityDuration','needBreakdown.baseline','needBreakdown.debt','needBreakdown.naps',
                'needBreakdown.strain','needBreakdown.total']
                for sleep_col in sleep_cols:
                    all_data['sleep.' + sleep_col]=all_data['sleep.' + sleep_col].astype(float).apply(lambda x: np.nan if np.isnan(x) else x/60000)

                self.all_data=all_data
                return all_data
        else:
            print("Please run the authorization function first")

    def get_activities_all(self):
        '''
        Activity data is pulled through the get_keydata functions so if the data pull is present, this function
        just transforms the activity column into a dataframe of activities, where each activity is a row.
        If it has not been pulled, this function runs the key data function then returns the activity dataframe'''
        
        if self.sport_dict:
            sport_dict=self.sport_dict
        else:
            sports=client.pull_api('https://api-7.whoop.com/sports')
            sport_dict={sport['id']:sport['name'] for sport in sports}
            self.sport_dict=self.sport_dict
        
        if self.start_datetime:
            ## process activity data
            
            if self.all_data is not None:
                ## use existing 
                data=self.all_data                
            else:
                ## pull all data to process activities
                data=self.get_keydata_all()
            ## now process activities data
            act_data=pd.json_normalize(data[data['strain.workouts'].apply(len)>0]['strain.workouts'].apply(lambda x: x[0]))
            act_data[['during.upper','during.lower']]=act_data[['during.upper','during.lower']].apply(pd.to_datetime)
            act_data['length']=(act_data['during.upper']-act_data['during.lower'])
            act_data['total_minutes']=act_data.length.apply(lambda x: x.total_seconds()/60.0)
            ## remember to drop length
            for z in range(0,6):
                 act_data['zone{}_minutes'.format(z+1)]=act_data['zones'].apply(lambda x: x[z]/60000.)
            act_data['sport_name']=activities.sportId.apply(lambda x: sport_dict[x])
            
            act_data['day']=act_data['during.lower'].dt.strftime('%Y-%m-%d')
            act_data.drop(['zones','during.bounds'],axis=1,inplace=True)
            act_data.drop_duplicates(inplace=True)
            self.all_activities=act_data
            return act_data  
        else:
            print("Please run the authorization function first")
    
    def get_activities_timeframe(self,start,end=datetime.strftime(datetime.utcnow(),"%Y-%m-%d")):
        '''
        Activity data is pulled through the get_keydata functions so if the data pull is present, this function
        just transforms the activity column into a dataframe of activities, where each activity is a row.
        If it has not been pulled, this function runs the key data function then returns the activity dataframe'''
        
        if self.auth_code:

            if self.sport_dict:
                sport_dict=self.sport_dict
            else:
                sports=client.pull_api('https://api-7.whoop.com/sports')
                sport_dict={sport['id']:sport['name'] for sport in sports}
                self.sport_dict=self.sport_dict

            ## process activity data
            if self.all_data is not None:
                ## use existing
                data=self.all_data
                data=data[(data.day>='start')&(data.day<='end')].copy(deep=True)
            else:
                ## pull timeframe data
                data=self.get_keydata_timeframe(start,end)
            ## now process activities data
            act_data=pd.json_normalize(data[data['strain.workouts'].apply(len)>0]['strain.workouts'].apply(lambda x: x[0]))
            act_data[['during.upper','during.lower']]=act_data[['during.upper','during.lower']].apply(pd.to_datetime)
            act_data['length']=(act_data['during.upper']-act_data['during.lower'])
            act_data['total_minutes']=act_data.length.apply(lambda x: x.total_seconds()/60.0)
            ## remember to drop length
            for z in range(0,6):
                 act_data['zone{}_minutes'.format(z+1)]=act_data['zones'].apply(lambda x: x[z]/60000.)
            act_data['sport_name']=activities.sportId.apply(lambda x: sport_dict[x])

            act_data['day']=act_data['during.lower'].dt.strftime('%Y-%m-%d')
            act_data.drop(['zones','during.bounds'],axis=1,inplace=True)
            act_data.drop_duplicates(inplace=True)
            self.all_activities=act_data
            return act_data  
        else:
            print("Please run the authorization function first")

    def get_keydata_timeframe(self,start,end=datetime.strftime(datetime.utcnow(),"%Y-%m-%d")):
        '''
        This function returns a dataframe of WHOOP metrics for each day in a specified time period. 
        To use this function, provide a start and end date in string format as follows "YYYY-MM-DD".
        If no end time is provided, it will default to today
        In the resulting dataframe, each day is a row and contains strain, recovery, and sleep information
        '''
        
        st=datetime.strptime(start,'%Y-%m-%d')
        e=datetime.strptime(end,'%Y-%m-%d')
        if st>e:
            if e>datetime.today():
                print("Please enter an end date earlier than tomorrow")
            else:
                print("Please enter a start date that is earlier than your end date")
        else:
            if self.auth_code:
                end_time='T23:59:59.999Z'
                start_time='T00:00:00.000Z'
                intervals=rrule.rrule(freq=WEEKLY,interval=1,until=e, dtstart=st)
                date_range=[[d.strftime('%Y-%m-%d') + start_time,
                            (d+relativedelta.relativedelta(weeks=1)).strftime('%Y-%m-%d') + end_time] for d in intervals if d<=e]
                time_data=pd.DataFrame()
                for dates in date_range:
                    cycle_url='https://api-7.whoop.com/users/{}/cycles?end={}&start={}'.format(self.whoop_id,
                                                                                           dates[1],
                                                                                           dates[0])
                    data=self.pull_api(cycle_url,df=True)
                    time_data=pd.concat([time_data,data])
                time_data.reset_index(drop=True,inplace=True)

                ## fixing the day column so it's not a list
                time_data['days']=time_data['days'].map(lambda d: d[0])
                time_data.rename(columns={"days":'day'},inplace=True)

                ## Putting all time into minutes instead of milliseconds
                sleep_cols=['qualityDuration','needBreakdown.baseline','needBreakdown.debt','needBreakdown.naps',
                'needBreakdown.strain','needBreakdown.total']
                for sleep_col in sleep_cols:
                    time_data['sleep.' + sleep_col]=time_data['sleep.' + sleep_col].astype(float).apply(lambda x: np.nan if np.isnan(x) else x/60000)

                return time_data
            else:
                print("Please run the authorization function first")

            

In [253]:
client=whoop_login()
client.get_authorization('whoop.ini')

In [240]:
activities=client.get_all_activities()

In [165]:
client=whoop_login()
client.get_authorization()

In [254]:
timed=client.get_keydata_timeframe("2020-08-01","2020-08-30")
timed.shape

(40, 35)

In [241]:
data=client.get_all_keydata()
data.head()

,day,id,lastUpdatedAt,predictedEnd,during.bounds,during.lower,during.upper,recovery.blackoutUntil,recovery.calibrating,recovery.heartRateVariabilityRmssd,recovery.id,recovery.responded,recovery.restingHeartRate,recovery.score,recovery.state,recovery.surveyResponseId,recovery.timestamp,sleep.id,sleep.naps,sleep.needBreakdown.baseline,sleep.needBreakdown.debt,sleep.needBreakdown.naps,sleep.needBreakdown.strain,sleep.needBreakdown.total,sleep.qualityDuration,sleep.score,sleep.sleeps,sleep.state,strain.averageHeartRate,strain.kilojoules,strain.maxHeartRate,strain.rawScore,strain.score,strain.state,strain.workouts,recovery,sleep.needBreakdown
0,2018-05-30,3001614,2018-05-31T11:45:38.237804+00:00,2018-05-31T04:00:00+00:00,[),2018-05-30T04:30:03.229+00:00,2018-05-31T03:58:58.144+00:00,None,True,0.061970,2778943.0,True,42.0,69.0,complete,4.73563e+06,2018-05-30T11:15:00+00:00,6460814.0,[],501.492433,0.000000,0.0,3.066450,504.558883,372.488367,74.0,"[{'cyclesCount': 2, 'disturbanceCount': 4, 'du...",complete,59,11196.0,166,0.005353,11.859412,complete,"[{'altitudeChange': None, 'altitudeGain': None...",NaN,NaN
1,2018-05-31,3017769,2018-06-01T12:00:16.753302+00:00,2018-06-01T03:58:58.144+00:00,[),2018-05-31T03:58:58.144+00:00,2018-06-01T04:00:48.207+00:00,None,True,0.098265,2787697.0,True,39.0,94.0,complete,4.74827e+06,2018-05-31T11:26:57.27+00:00,6479679.0,[],464.954850,60.444117,0.0,19.087683,544.486683,391.471567,72.0,"[{'cyclesCount': 3, 'disturbanceCount': 10, 'd...",complete,58,15086.0,175,0.013025,15.908476,complete,"[{'altitudeChange': None, 'altitudeGain': None...",NaN,NaN
2,2018-06-01,3034067,2018-06-02T14:35:44.795092+00:00,2018-06-02T03:58:58.144+00:00,[),2018-06-01T04:00:48.207+00:00,2018-06-02T05:30:46.683+00:00,None,True,0.075517,2796682.0,True,42.0,75.0,complete,4.76646e+06,2018-06-01T11:46:48.271+00:00,6498229.0,[],464.949833,65.092967,0.0,43.111283,573.154117,433.986350,76.0,"[{'cyclesCount': 4, 'disturbanceCount': 6, 'du...",complete,60,13594.0,157,0.006016,12.424925,None,[],NaN,NaN
3,2018-06-02,3053216,2018-06-03T12:54:58.704101+00:00,2018-06-03T04:00:48.207+00:00,[),2018-06-02T05:30:46.683+00:00,2018-06-03T06:10:42.099+00:00,None,False,0.060051,2807267.0,True,39.0,61.0,complete,4.78197e+06,2018-06-02T14:30:56+00:00,6519662.0,[],464.944817,71.734333,0.0,21.722100,558.401267,448.963567,80.0,"[{'cyclesCount': 2, 'disturbanceCount': 6, 'du...",complete,62,17213.0,169,0.014860,16.579523,complete,"[{'altitudeChange': None, 'altitudeGain': None...",NaN,NaN
4,2018-06-03,3067760,2018-06-04T12:02:27.424057+00:00,2018-06-04T05:30:46.683+00:00,[),2018-06-03T06:10:42.099+00:00,2018-06-04T04:46:09.29+00:00,None,False,0.146829,2813899.0,True,38.0,94.0,complete,4.79414e+06,2018-06-03T16:10:10.913+00:00,6534354.0,[],464.939817,59.777967,0.0,47.940200,572.657983,411.987683,72.0,"[{'cyclesCount': 2, 'disturbanceCount': 4, 'du...",complete,54,14532.0,169,0.019051,17.785046,complete,"[{'altitudeChange': None, 'altitudeGain': None...",NaN,NaN


In [231]:
not data.empty

True

In [215]:
prac=None
if prac:
    print("y")

In [143]:
pd.options.display.max_columns=100
data-['strain.workouts'].apply(lambda x: x[0])

IndexError: list index out of range

In [163]:
data_store=client.time_data
data_store.head()
data_store.days.max(),data_store.days.min()
activities=pd.json_normalize(data_store[data_store['strain.workouts'].apply(len)>0]['strain.workouts'].apply(lambda x: x[0]))
activities['length']=(pd.to_datetime(activities['during.upper'])-pd.to_datetime(activities['during.lower']))
activities['total_minutes']=activities.length.apply(lambda x: x.total_seconds()/60.0)
for z in range(0,6):
     activities['min_zone{}'.format(z+1)]=activities['zones'].apply(lambda x: x[z]/60000.)
activities.head()
activities[['during.upper','during.lower']]=activities[['during.upper','during.lower']].apply(pd.to_datetime)
activities

,altitudeChange,altitudeGain,averageHeartRate,cumulativeWorkoutStrain,distance,gpsEnabled,id,kilojoules,maxHeartRate,rawScore,responded,score,source,sportId,state,surveyResponseId,timezoneOffset,zones,during.bounds,during.lower,during.upper,length,total_minutes,min_zone1,min_zone2,min_zone3,min_zone4,min_zone5,min_zone6
0,None,None,152,16.0264,None,False,98057571,3588.14,178,0.013330,True,16.026390,auto,0,complete,48948968.0,-0400,"[25992, 11535, 384524, 1072806, 2066819, 145148]",[),2020-08-01 20:59:20.422000+00:00,2020-08-01 22:01:07.207000+00:00,01:01:46.785000,61.779750,0.433200,0.192250,6.408733,17.880100,34.446983,2.419133
1,None,None,136,16.3354,None,False,99285290,5368.95,171,0.014164,True,16.335431,user,52,complete,49139696.0,-0700,"[0, 644096, 2121607, 2770482, 1583264, 0]",[),2020-08-05 00:38:12.219000+00:00,2020-08-05 02:36:51.629000+00:00,01:58:39.410000,118.656833,0.000000,10.734933,35.360117,46.174700,26.387733,0.000000
2,None,None,134,13.8227,None,False,102062483,3088.02,175,0.008412,True,13.822746,auto,1,complete,49568095.0,-0700,"[188457, 318188, 1440032, 1361208, 908431, 32684]",[),2020-08-12 00:17:11.902000+00:00,2020-08-12 01:28:00.863000+00:00,01:10:48.961000,70.816017,3.140950,5.303133,24.000533,22.686800,15.140517,0.544733
3,None,None,131,9.4889,None,False,104729259,1389.94,162,0.003306,True,9.488939,auto,1,complete,50007962.0,-0700,"[0, 296117, 504697, 964183, 227823, 0]",[),2020-08-19 01:56:13.340000+00:00,2020-08-19 02:29:26.121000+00:00,00:33:12.781000,33.213017,0.000000,4.935283,8.411617,16.069717,3.797050,0.000000
4,None,None,139,12.8117,None,False,106271857,2222.67,169,0.006564,True,12.811659,user,17,complete,50248668.0,-0700,"[2883, 261518, 408552, 1247720, 836335, 0]",[),2020-08-23 00:56:38.215000+00:00,2020-08-23 01:42:35.184000+00:00,00:45:56.969000,45.949483,0.048050,4.358633,6.809200,20.795333,13.938917,0.000000
5,None,None,139,12.8117,None,False,106271857,2222.67,169,0.006564,True,12.811659,user,17,complete,50248668.0,-0700,"[2883, 261518, 408552, 1247720, 836335, 0]",[),2020-08-23 00:56:38.215000+00:00,2020-08-23 01:42:35.184000+00:00,00:45:56.969000,45.949483,0.048050,4.358633,6.809200,20.795333,13.938917,0.000000
6,None,None,117,13.3261,None,False,106594391,2691.47,148,0.004413,False,10.884312,auto,52,complete,NaN,-0700,"[0, 1511157, 3461695, 267250, 0, 0]",[),2020-08-23 19:12:10.160000+00:00,2020-08-23 20:39:30.223000+00:00,01:27:20.063000,87.334383,0.000000,25.185950,57.694917,4.454167,0.000000,0.000000
7,None,None,152,13.0958,None,False,107036810,1989.36,173,0.007051,True,13.095841,user,82,complete,50369159.0,-0700,"[0, 1961, 126890, 810375, 1097813, 15383]",[),2020-08-25 01:40:47.273000+00:00,2020-08-25 02:14:59.656000+00:00,00:34:12.383000,34.206383,0.000000,0.032683,2.114833,13.506250,18.296883,0.256383
8,None,None,124,9.0430,None,False,107857025,1391.51,163,0.002990,True,9.043038,auto,1,complete,50508341.0,-0700,"[119204, 280738, 1101648, 617163, 173032, 0]",[),2020-08-27 02:37:43.320000+00:00,2020-08-27 03:15:55.066000+00:00,00:38:11.746000,38.195767,1.986733,4.678967,18.360800,10.286050,2.883867,0.000000
9,None,None,122,13.4584,None,False,109410399,1550.44,146,0.002769,False,8.721113,auto,52,complete,NaN,-0700,"[0, 504681, 1700595, 496985, 0, 0]",[),2020-08-30 18:37:26.305000+00:00,2020-08-30 19:22:28.527000+00:00,00:45:02.222000,45.037033,0.000000,8.411350,28.343250,8.283083,0.000000,0.000000


In [251]:
intervals=rrule.rrule(freq=WEEKLY,
                      interval=1,
                      until=datetime.strptime("2020-08-30","%Y-%m-%d"),
                      dtstart=datetime.strptime("2020-08-01","%Y-%m-%d"))
[n for n in intervals]

[datetime.datetime(2020, 8, 1, 0, 0),
 datetime.datetime(2020, 8, 8, 0, 0),
 datetime.datetime(2020, 8, 15, 0, 0),
 datetime.datetime(2020, 8, 22, 0, 0),
 datetime.datetime(2020, 8, 29, 0, 0)]

In [119]:
datetime.strptime("2020-08-05","%Y-%m-%d")

datetime.datetime(2020, 8, 5, 0, 0)

In [112]:
test=parser.isoparse(client.start_datetime).replace(tzinfo=None)
intervals=rrule.rrule(freq=WEEKLY,interval=1,until=datetime.utcnow(), dtstart=test)
date_range=[[d.strftime('%Y-%m-%d'),
                        (d+relativedelta.relativedelta(weeks=1)).strftime('%Y-%m-%d')] for d in intervals]
            
for dates in date_range:
    cycle_url='https://api-7.whoop.com/users/cycles?end={}&start={}'.format(
                                                                           dates[1],
                                                                           dates[0])
    print(cycle_url)

https://api-7.whoop.com/users/cycles?end=2018-06-06&start=2018-05-30
https://api-7.whoop.com/users/cycles?end=2018-06-13&start=2018-06-06
https://api-7.whoop.com/users/cycles?end=2018-06-20&start=2018-06-13
https://api-7.whoop.com/users/cycles?end=2018-06-27&start=2018-06-20
https://api-7.whoop.com/users/cycles?end=2018-07-04&start=2018-06-27
https://api-7.whoop.com/users/cycles?end=2018-07-11&start=2018-07-04
https://api-7.whoop.com/users/cycles?end=2018-07-18&start=2018-07-11
https://api-7.whoop.com/users/cycles?end=2018-07-25&start=2018-07-18
https://api-7.whoop.com/users/cycles?end=2018-08-01&start=2018-07-25
https://api-7.whoop.com/users/cycles?end=2018-08-08&start=2018-08-01
https://api-7.whoop.com/users/cycles?end=2018-08-15&start=2018-08-08
https://api-7.whoop.com/users/cycles?end=2018-08-22&start=2018-08-15
https://api-7.whoop.com/users/cycles?end=2018-08-29&start=2018-08-22
https://api-7.whoop.com/users/cycles?end=2018-09-05&start=2018-08-29
https://api-7.whoop.com/users/cycl

In [158]:
sports=client.pull_api('https://api-7.whoop.com/sports')
{sport['id']:sport['name'] for sport in sports}

{-1: 'Activity',
 0: 'Running',
 1: 'Cycling',
 16: 'Baseball',
 17: 'Basketball',
 18: 'Rowing',
 19: 'Fencing',
 20: 'Field Hockey',
 21: 'Football',
 22: 'Golf',
 24: 'Ice Hockey',
 25: 'Lacrosse',
 27: 'Rugby',
 28: 'Sailing',
 29: 'Skiing',
 30: 'Soccer',
 31: 'Softball',
 32: 'Squash',
 33: 'Swimming',
 34: 'Tennis',
 35: 'Track & Field',
 36: 'Volleyball',
 37: 'Water Polo',
 38: 'Wrestling',
 39: 'Boxing',
 42: 'Dance',
 43: 'Pilates',
 44: 'Yoga',
 45: 'Weightlifting',
 46: 'Canoeing',
 47: 'Cross Country Skiing',
 48: 'Functional Fitness',
 49: 'Duathlon',
 50: 'Machine Workout',
 51: 'Gymnastics',
 52: 'Hiking/Rucking',
 53: 'Horseback Riding',
 54: 'Jogging',
 55: 'Kayaking',
 56: 'Martial Arts',
 57: 'Mountain Biking',
 58: 'Obstacle Racing',
 59: 'Powerlifting',
 60: 'Rock Climbing',
 61: 'Paddleboarding',
 62: 'Triathlon',
 63: 'Walking',
 64: 'Surfing',
 65: 'Elliptical',
 66: 'Stairmaster',
 67: 'Plyometrics',
 68: 'Spinning',
 69: 'Sex',
 70: 'Meditation',
 71: 'Other

In [108]:
st='2020-08-04'
e='2021-8-1'
st=datetime.strptime(st,'%Y-%m-%d')
e=datetime.strptime(e,'%Y-%m-%d')
if e>st:
    if e>datetime.today():
        print('in future')    
    else:
        print('good')
else:
    print(st,e)

in future


In [73]:
class test_class:
    
    def __init__(self):
        pass
    

In [74]:
t=test_class


In [75]:
t.test='ta'
t.test


'ta'

In [224]:
iso_date='2018-05-19T00:30:04.262Z'
yourdate = parser.isoparse(iso_date) #+ relativedelta.relativedelta(weeks=1)
datetime.utcnow(), yourdate

(datetime.datetime(2020, 8, 19, 6, 35, 11, 750700),
 datetime.datetime(2018, 5, 19, 0, 30, 4, 262000, tzinfo=tzutc()))

In [225]:
datetime.utcnow().isoformat()

'2020-08-19T06:35:12.064616'

In [226]:
yourdate.replace(hour=0,minute=0, second=0,microsecond=0,tzinfo=None)

datetime.datetime(2018, 5, 19, 0, 0)

In [227]:
datetime.combine(yourdate.date(),), yourdate.date,yourdate.replace(tzinfo=None)

TypeError: Required argument 'time' (pos 2) not found

In [228]:
intervals=rrule.rrule(freq=WEEKLY,interval=1,until=datetime.utcnow(), dtstart=yourdate.replace(tzinfo=None))
date_range=[[d.strftime('%Y-%m-%d') + 'T00:00:00.000Z',
  (d+relativedelta.relativedelta(weeks=1)).strftime('%Y-%m-%d') + 'T00:00:00.000Z'] for d in intervals]
date_range

[['2018-05-19T00:00:00.000Z', '2018-05-26T00:00:00.000Z'],
 ['2018-05-26T00:00:00.000Z', '2018-06-02T00:00:00.000Z'],
 ['2018-06-02T00:00:00.000Z', '2018-06-09T00:00:00.000Z'],
 ['2018-06-09T00:00:00.000Z', '2018-06-16T00:00:00.000Z'],
 ['2018-06-16T00:00:00.000Z', '2018-06-23T00:00:00.000Z'],
 ['2018-06-23T00:00:00.000Z', '2018-06-30T00:00:00.000Z'],
 ['2018-06-30T00:00:00.000Z', '2018-07-07T00:00:00.000Z'],
 ['2018-07-07T00:00:00.000Z', '2018-07-14T00:00:00.000Z'],
 ['2018-07-14T00:00:00.000Z', '2018-07-21T00:00:00.000Z'],
 ['2018-07-21T00:00:00.000Z', '2018-07-28T00:00:00.000Z'],
 ['2018-07-28T00:00:00.000Z', '2018-08-04T00:00:00.000Z'],
 ['2018-08-04T00:00:00.000Z', '2018-08-11T00:00:00.000Z'],
 ['2018-08-11T00:00:00.000Z', '2018-08-18T00:00:00.000Z'],
 ['2018-08-18T00:00:00.000Z', '2018-08-25T00:00:00.000Z'],
 ['2018-08-25T00:00:00.000Z', '2018-09-01T00:00:00.000Z'],
 ['2018-09-01T00:00:00.000Z', '2018-09-08T00:00:00.000Z'],
 ['2018-09-08T00:00:00.000Z', '2018-09-15T00:00:00.000Z'

In [89]:
date_rng=pd.date_range('2020-03-27','2020-06-01',freq='W',).to_list()
dates=[d for d in date_rng]
dates

AttributeError: 'str' object has no attribute 'toordinal'

In [ ]:
def get_auth_code(user_entry=True):
    '''Function to get a user's authorization code
    if user_

In [28]:
config = configparser.ConfigParser()
config.read("../strava.ini")
whoop_user=config['whoop']['username']
whoop_pass=config['whoop']['pass']
whoop_id=config['whoop']['userid']

In [263]:
w='https://app.whoop.com/login'

In [264]:
browser=webdriver.Chrome('/usr/local/bin/chromedriver')
browser.get(w)

In [255]:
username=browser.find_element_by_name('username')
password=browser.find_element_by_name('password')
username.send_keys(whoop_user)
password.send_keys(whoop_pass)
submit=browser.find_element_by_name('form')
submit.click()

In [33]:
whoop_requests=[request for request in browser.requests if 'w' in request.url]
auth_code=whoop_requests[0].headers['authorization']

In [91]:
def pull_api(u,bearer=auth_code,df=True,):
    headers={'authorization':bearer}
    pull=requests.get(u,headers=headers)
    if pull.status_code==200 and len(pull.content)>1:
        if df:
            d=pd.json_normalize(pull.json())
            return d
        else:
            return pull.json()
    else:
        return "no response"

timezoneOffset': '-0700'

In [181]:
## works even if the date is off!
time_test='https://api-7.whoop.com/users/24590/cycles?end=2018-05-31T23:59:59.999-07:00&start=2018-05-25T00:00:00.000-07:00'.format(end,start)
pull_api(time_test,df=False)

[{'days': ['2018-05-29'],
  'during': {'bounds': '[)',
   'lower': '2018-05-29T04:00:00+00:00',
   'upper': '2018-05-30T04:30:03.229+00:00'},
  'id': 2993303,
  'lastUpdatedAt': '2018-05-30T11:37:31.755759+00:00',
  'predictedEnd': '2018-05-30T04:00:00+00:00',
  'recovery': None,
  'sleep': {'id': None,
   'naps': [],
   'needBreakdown': None,
   'qualityDuration': None,
   'score': None,
   'sleeps': [],
   'state': None},
  'strain': {'averageHeartRate': 75,
   'kilojoules': 1932,
   'maxHeartRate': 151,
   'rawScore': 0.000476343479643741,
   'score': 4.84126152305615,
   'state': None,
   'workouts': []}},
 {'days': ['2018-05-30'],
  'during': {'bounds': '[)',
   'lower': '2018-05-30T04:30:03.229+00:00',
   'upper': '2018-05-31T03:58:58.144+00:00'},
  'id': 3001614,
  'lastUpdatedAt': '2018-05-31T11:45:38.237804+00:00',
  'predictedEnd': '2018-05-31T04:00:00+00:00',
  'recovery': {'blackoutUntil': None,
   'calibrating': True,
   'heartRateVariabilityRmssd': 0.0619698,
   'id': 277

In [36]:
all_data=pd.DataFrame()
for i in range(len(dates)-1):
    start=dates[i]
    end=dates[i+1]
    u='https://api-7.whoop.com/users/24590/cycles?end={}T23:59:59.999Z&start={}T00:00:00.000Z'.format(end,start)
    act_df=pull_api(u)
    all_data=pd.concat([all_data,act_df])
all_data.reset_index(drop=True,inplace=True)

In [37]:
all_data

,days,id,lastUpdatedAt,predictedEnd,during.bounds,during.lower,during.upper,recovery.blackoutUntil,recovery.calibrating,recovery.heartRateVariabilityRmssd,recovery.id,recovery.responded,recovery.restingHeartRate,recovery.score,recovery.state,recovery.surveyResponseId,recovery.timestamp,sleep.id,sleep.naps,sleep.needBreakdown.baseline,sleep.needBreakdown.debt,sleep.needBreakdown.naps,sleep.needBreakdown.strain,sleep.needBreakdown.total,sleep.qualityDuration,sleep.score,sleep.sleeps,sleep.state,strain.averageHeartRate,strain.kilojoules,strain.maxHeartRate,strain.rawScore,strain.score,strain.state,strain.workouts
0,[2020-03-29],25965683,2020-03-30T13:14:57.652058+00:00,2020-03-30T04:25:00+00:00,[),2020-03-29T04:25:00+00:00,2020-03-30T03:48:05.723+00:00,None,False,0.074748,26975073,False,38,41,complete,None,2020-03-29T14:24:00+00:00,59188595,[],27712766,0,3242760,900639,25370645,30390372,100,"[{'cyclesCount': 7, 'disturbanceCount': 21, 'd...",complete,68,15226.50,168,0.025447,19.124589,complete,"[{'altitudeChange': None, 'altitudeGain': None..."
1,[2020-03-30],26043770,2020-03-31T11:03:57.003419+00:00,2020-03-31T03:48:05.723+00:00,[),2020-03-30T03:48:05.723+00:00,2020-03-31T04:57:42.098+00:00,None,False,0.062112,27060531,False,41,40,complete,None,2020-03-30T13:11:00+00:00,59377072,[],27712513,0,0,3961243,31673756,30384437,96,"[{'cyclesCount': 3, 'disturbanceCount': 16, 'd...",complete,56,9825.13,161,0.005344,11.851660,complete,"[{'altitudeChange': None, 'altitudeGain': None..."
2,[2020-03-31],26140353,2020-04-01T12:43:11.805959+00:00,2020-04-01T03:48:05.723+00:00,[),2020-03-31T04:57:42.098+00:00,2020-04-01T02:45:37.582+00:00,None,False,0.090682,27161155,False,38,60,complete,None,2020-03-31T10:59:57.164+00:00,59602796,[],27712260,712600,0,1143201,29568062,20078533,68,"[{'cyclesCount': 3, 'disturbanceCount': 9, 'du...",complete,61,9055.14,181,0.012214,15.583785,complete,"[{'altitudeChange': None, 'altitudeGain': None..."
3,[2020-04-01],26249269,2020-04-02T12:01:45.273664+00:00,2020-04-02T02:45:37.582+00:00,[),2020-04-01T02:45:37.582+00:00,2020-04-02T04:28:55.27+00:00,None,False,0.121504,27275378,False,40,86,complete,None,2020-04-01T12:23:00+00:00,59855084,[],27712007,4433921,0,2449222,34595150,31014046,90,"[{'cyclesCount': 5, 'disturbanceCount': 19, 'd...",complete,61,10750.80,173,0.009339,14.288825,complete,"[{'altitudeChange': None, 'altitudeGain': None..."
4,[2020-04-02],26337359,2020-04-03T11:08:50.773869+00:00,2020-04-03T02:45:37.582+00:00,[),2020-04-02T04:28:55.27+00:00,2020-04-03T03:20:44.926+00:00,None,False,0.060937,27367765,False,46,30,complete,None,2020-04-02T11:50:00+00:00,60071256,"[{'cyclesCount': 0, 'disturbanceCount': 1, 'du...",27711754,1781469,0,1930739,31423963,23796879,76,"[{'cyclesCount': 1, 'disturbanceCount': 6, 'du...",complete,59,9364.09,145,0.004101,10.516768,None,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,[2020-05-27],32400095,2020-05-28T12:44:10.737969+00:00,2020-05-28T03:52:43.715+00:00,[),2020-05-27T03:52:43.715+00:00,2020-05-28T05:17:29.016+00:00,None,False,0.075225,33780765,False,39,48,complete,None,2020-05-27T12:21:55+00:00,75082737,[],27697983,3877717,0,2566693,34142394,27472766,80,"[{'cyclesCount': 7, 'disturbanceCount': 10, 'd...",complete,58,10574.70,146,0.004620,11.116088,None,[]
68,[2020-05-28],32497836,2020-05-29T12:40:40.330552+00:00,2020-05-29T03:52:43.715+00:00,[),2020-05-28T05:17:29.016+00:00,2020-05-29T03:54:42.316+00:00,None,False,0.060267,33883561,False,42,28,complete,None,2020-05-28T12:32:00+00:00,75407141,[],27697734,3561137,0,960518,32219390,21353974,66,"[{'cyclesCount': 4, 'disturbanceCount': 6, 'du...",complete,57,10005.50,157,0.005817,12.266130,complete,"[{'altitudeChange': None, 'altitudeGain': None..."
69,[2020-05-29],32622198,2020-05-30T09:37:18.851695+00:00,2020-05-30T03:54:42.316+00:00,[),2020-05-29T03:54:42.316+00:00,2020-05-30T03:41:00+00:00,None,False,0.069490,3401276

In [14]:
[request.url for request in browser.requests if 'api' in request.url]

['https://www.google-analytics.com/r/collect?v=1&_v=j83&a=955321665&t=pageview&_s=1&dl=https%3A%2F%2Fapp.whoop.com%2Flogin&dp=%2Flogin%3Fapi%26for&ul=en-us&de=UTF-8&dt=Login&sd=30-bit&sr=1440x900&vp=1200x645&je=0&_u=aGBAAEADQ~&jid=1428063794&gjid=799478635&cid=1701259047.1597610646&tid=UA-64891632-3&_gid=2104853456.1597610646&_r=1&cd4=https%3A%2F%2Fapp.whoop.com%2Flogin&z=770047105',
 'https://api-7.whoop.com/oauth/token',
 'https://api-7.whoop.com/users/24590?include=profile&include=teams',
 'https://api-7.whoop.com/users/24590?include=profile&include=teams',
 'https://api-7.whoop.com/users/24590',
 'https://api-7.whoop.com/users/24590/cycles?end=2020-08-17T06:59:59.999Z&start=2020-08-15T07:00:00.000Z',
 'https://api-7.whoop.com/sports',
 'https://api-7.whoop.com/voice-of-whoop',
 'https://api-7.whoop.com/voice-of-whoop',
 'https://api-7.whoop.com/users/24590/cycles?end=2020-08-15T06:59:59.999Z&start=2020-08-05T07:00:00.000Z',
 'https://api-7.whoop.com/users/24590/cycles?end=2020-09-0

In [20]:
pd.options.display.max_columns=100

In [92]:
pull_api('https://api-7.whoop.com/users/{}?include=profile&include=teams'.format(whoop_id),df=False)

{'adminDivision': 'DC',
 'avatarUrl': 'https://s3-us-west-2.amazonaws.com/avatars.whoop.com/uploads/uploads/user/24550-24599/24590_5b64ee5ac5c254a5123b6e1bcdbc7edb.jpeg',
 'city': 'Washington',
 'concealed': False,
 'country': 'US',
 'createdAt': '2018-05-21T00:44:06.208Z',
 'email': 'irarickman@gmail.com',
 'firstName': 'Ira',
 'fullName': 'Ira Rickman',
 'id': 24590,
 'lastName': 'Rickman',
 'membershipStatus': 'trialing',
 'preferences': {'performanceOptimizationAssessment': True,
  'performanceOptimizationDayOfWeek': 1},
 'privacyProfile': {'comps': 'all',
  'intensity': 'all',
  'overview': 'all',
  'recovery': 'all',
  'sleep': 'all',
  'stats': 'all'},
 'profile': {'avgHeartRate': None,
  'bioDataId': 10339530,
  'birthday': '1992-08-05T00:00:00.000Z',
  'canUploadData': True,
  'createdAt': '2018-05-30T00:30:04.262Z',
  'fitnessLevel': 'recreational_enthusiast',
  'gender': 'male',
  'height': 1.7526,
  'id': 10339530,
  'kilojoules': None,
  'maxHeartRate': 192,
  'minHeartRat

In [46]:
datetime.utcnow().isoformat()[:-3] + "Z"

'2020-08-19T01:48:53.014Z'

In [48]:
datetime('2018-05-30T00:30:04.262Z')

TypeError: an integer is required (got type str)

## Old Way

In [23]:
strain_click=browser.find_elements_by_class_name('score')[0]
strain_click.click()

In [23]:
strain_amt=browser.find_element_by_class_name("workout-strain")
strain_amt.get_attribute('innerHTML')
strain_amt.find_element_by_tag_name("strong").text

'16.2'

In [24]:
buttons=browser.find_elements_by_tag_name('button')
#[x for x in [span.find_elements_by_tag_name('span')  
[x.text for item in [x for x in [span.find_elements_by_tag_name('span') for span in [b for b in buttons if b.get_attribute("ng-click")=="click($event, activity.id)"]]] for x in item]


['Running', '16.2']

In [25]:
for x in buttons:
    if x.get_attribute("ng-click")=="click($event, activity.id)":
        spans=x.find_elements_by_tag_name("span")
        print([x.text for x in spans], x)
    else:
        print("nothing", x)

nothing <selenium.webdriver.remote.webelement.WebElement (session="08fde42d8d6815291fceb7d2092c9f08", element="0.7561134548339705-10")>
nothing <selenium.webdriver.remote.webelement.WebElement (session="08fde42d8d6815291fceb7d2092c9f08", element="0.7561134548339705-11")>
['Running', '16.2'] <selenium.webdriver.remote.webelement.WebElement (session="08fde42d8d6815291fceb7d2092c9f08", element="0.7561134548339705-12")>
nothing <selenium.webdriver.remote.webelement.WebElement (session="08fde42d8d6815291fceb7d2092c9f08", element="0.7561134548339705-13")>
nothing <selenium.webdriver.remote.webelement.WebElement (session="08fde42d8d6815291fceb7d2092c9f08", element="0.7561134548339705-14")>
nothing <selenium.webdriver.remote.webelement.WebElement (session="08fde42d8d6815291fceb7d2092c9f08", element="0.7561134548339705-15")>
nothing <selenium.webdriver.remote.webelement.WebElement (session="08fde42d8d6815291fceb7d2092c9f08", element="0.7561134548339705-16")>
nothing <selenium.webdriver.remote.w

In [24]:
def go_to_strain():
    work=0
    try:
        strain_click=browser.find_elements_by_class_name('score')[0]
        strain_click.click()
        work=1
    except StaleElementReferenceException:
        work=0

In [25]:
## goal function is to look for the activity - if it doesn't exist, then look for "no activity" 
## store the result, if the old activity strain is the same as the new

def get_activity_strain(old_strain,old_date):
    work=0
    while work==0:
        try:
            buttons=browser.find_elements_by_tag_name('button')
            new_strain=[x.text for item in [x for x in [span.find_elements_by_tag_name('span') for span in [b for b in buttons if b.get_attribute("ng-click")=="click($event, activity.id)"]]] for x in item]
            if len(new_strain)==0:
                new_strains=["null"]*6
            elif len(new_strain)<7:
                new_strains=new_strain+['null']*(6-len(new_strain))
            else:
                new_strains=new_strain
            if old_strain!=new_strains:
                return new_strains
                work=1
            elif old_date!=get_date(old_date):
                return new_strains
                work=1
        except StaleElementReferencException:
            work=0

In [26]:
def get_date(old_date):
    work=0
    while work==0:
        try:
            new_day=browser.find_element_by_class_name('datepicker--label')
            new_date=new_day.text
            if new_date!=old_date:
                return [new_date]
                work=1
        except StaleElementReferenceException:
            work=0

In [27]:
def get_scores(old_scores):
    work=0
    while work==0:
        try:
            new_scores=browser.find_elements_by_class_name('score')
            new_score_list=[x.text for x in new_scores]
            if new_score_list!=old_scores:
                return new_score_list
                work=1
        except StaleElementReferenceException:
            work=0
            

In [28]:
def get_button():
    work=0
    while work==0:
        try: 
            back=browser.find_element_by_class_name('datepicker--prev')
            back.click()
            work=1
        except StaleElementReferenceException:
            work=0

In [38]:
num=1
(get_date('today')[0].strip().lower()!='wed, may 30th' and num<300)

False

In [46]:
whoop=pd.DataFrame(columns=['strain','recovery','sleep_perf','sleep','rec_sleep','date', 
                            'activity_1','activity_1_score','activity_2','activity_2_score',
                           'activity_3','activity_3_score'])
num=0
old_date='today'
old_scores=['none']
old_strain=['null']
go_to_strain()
new_date=get_date(old_date)
while (new_date[0].strip().lower()!='wed, may 30th' and num<300):
    new_date=get_date(old_date)
    new_score=get_scores(old_scores)
    strains=get_activity_strain(old_strain,old_date)
    old_date=new_date
    old_scores=new_score
    old_strain=strains
    whoop.loc[len(whoop)]=new_score[:5]+new_date+strains
    get_button()
    num+=1

In [48]:
whoop.to_csv('../Data/whoop_data.csv',index=False)